In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
class FERDataset(Dataset):
    def __init__(self, csv_file, usage, transform=None):
        self.data = pd.read_csv(csv_file)
        self.data = self.data[self.data[' Usage'] == usage]  # Filter by usage (Training, PublicTest, PrivateTest)
        self.transform = transform
        self.data = self.data[self.data[' pixels'].apply(lambda x: len(x.split()) == 2304)]  # Remove invalid rows

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixel_values = self.data.iloc[idx, 2].split()
        pixels = np.array(pixel_values, dtype=np.uint8).reshape(48, 48)
        image = Image.fromarray(pixels).convert('RGB')
        label = int(self.data.iloc[idx, 0])
        if self.transform:
            image = self.transform(image)
        return image, label


In [3]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [9]:
# Load FER dataset (adjust path as needed)
train_dataset = FERDataset(csv_file='/content/fer+ck+augmentedfer.csv', usage='Training', transform=transform)
test_dataset = FERDataset(csv_file='/content/fer+ck+augmentedfer.csv', usage='PublicTest', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# Print dataset sizes
print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 11003
Test set size: 0


In [ ]:
device

device(type='cuda')

In [ ]:
# Load ResNet-18 model
model = resnet18(pretrained=False, num_classes=7)  # FER has 7 classes
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Define learning rate schedule
lr_stages = [10, 15]  # Epochs where LR changes
lr_values = [0.001, 0.0001, 0.00001]

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    for i, stage in enumerate(lr_stages):
        if epoch < stage:
            new_lr = lr_values[i]
            break
    else:
        new_lr = lr_values[-1]
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr

In [ ]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    adjust_learning_rate(optimizer, epoch)
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, LR: {optimizer.param_groups[0]['lr']}")

print("Training complete!")

Epoch 1, Loss: 1.4245, LR: 0.001
Epoch 2, Loss: 1.0537, LR: 0.001
Epoch 3, Loss: 0.8641, LR: 0.001
Epoch 4, Loss: 0.6368, LR: 0.001
Epoch 5, Loss: 0.3693, LR: 0.001
Epoch 6, Loss: 0.2063, LR: 0.001
Epoch 7, Loss: 0.1406, LR: 0.001
Epoch 8, Loss: 0.1073, LR: 0.001
Epoch 9, Loss: 0.0938, LR: 0.001
Epoch 10, Loss: 0.0768, LR: 0.001
Epoch 11, Loss: 0.0259, LR: 0.0001
Epoch 12, Loss: 0.0100, LR: 0.0001
Epoch 13, Loss: 0.0082, LR: 0.0001
Epoch 14, Loss: 0.0077, LR: 0.0001
Epoch 15, Loss: 0.0066, LR: 0.0001
Epoch 16, Loss: 0.0050, LR: 1e-05
Epoch 17, Loss: 0.0046, LR: 1e-05
Epoch 18, Loss: 0.0045, LR: 1e-05
Epoch 19, Loss: 0.0042, LR: 1e-05
Epoch 20, Loss: 0.0042, LR: 1e-05
Training complete!


In [12]:
def load_model(model_path):
    model = resnet18(pretrained=False, num_classes=7)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    print("Model loaded successfully!")
    return model

model = load_model('/content/resnet18_with_stepdecay_20.pth')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

evaluate(model, test_loader)

Test Accuracy: 99.64%


In [8]:
# Display some sample images with predictions
def show_sample_images(model, test_loader):
    model.eval()
    images, labels = next(iter(test_loader))
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    images = images.cpu().numpy()
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for i in range(5):
        img = np.transpose(images[i], (1, 2, 0)).squeeze()
        img = img * 0.5 + 0.5  # Unnormalize
        img = np.clip(img, 0, 1)
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(f'Pred: {predicted[i].item()}\nTrue: {labels[i].item()}')
        axes[i].axis('off')
    plt.show()

In [7]:
show_sample_images(model, test_loader)

NameError: name 'show_sample_images' is not defined

In [ ]:
model_path = "resnet18_with_stepdecay_20.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to resnet18_with_stepdecay_20.pth
